In [2]:
%pip install mxnet

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 41.1 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install gluonts

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 101.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 104.2 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [390]:
import time
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import glob
import random

In [391]:
from gluonts.model import deepar
from gluonts.dataset.common import ListDataset
from gluonts.evaluation import make_evaluation_predictions

In [392]:
from joblib import Parallel, delayed
import contextlib
import joblib
from tqdm import tqdm
import pickle

In [393]:
import scipy
import gc

In [394]:
np.random.seed(54321)
random.seed(54321)

# Read in Data and Prepare for Modeling

In [395]:
# Read in all csv files for England from the Processed directory and save in a list

# Create an empty list
total_df_list = list()

# Loop through all file names in alphebtical order to match R list.files function, 
# read into a pandas df, ensure each df is in chronological order, and append to list
for fname in sorted(glob.glob("Data/Unseen Sensor/Processed/*.csv")):
    print("Reading {}".format(fname))
    df = pd.read_csv(fname)
    df = df.sort_values(by="timestamp")
    total_df_list.append(df)

Reading Data/Unseen Sensor/Processed/A19-9336-1_Northbound_2019_Processed.csv
Reading Data/Unseen Sensor/Processed/A66-9521-1_Westbound_Processed.csv
Reading Data/Unseen Sensor/Processed/M40-7048-2_Southbound_Processed.csv
Reading Data/Unseen Sensor/Processed/M62-2056A_Eastbound_Processed.csv


In [396]:
# Read in the start and end points csv, and subtract 1 to deal with index differences between R and python
start_end = pd.read_csv("unseen_sensor_start_end_points.csv")
start_end["start"] = start_end["start"] - 1
start_end["end"] = start_end["end"]
start_end = start_end.drop(columns=['Unnamed: 0'])

In [397]:
# Initialize an empty list
subset_df_list = list()

# For each df in the total_df_list
for idx, df in enumerate(total_df_list):
    
    # subset the df using the start and end points
    subset_df = df.iloc[start_end.iloc[idx,0]:start_end.iloc[idx,1], ]\
    .reset_index(drop=True).reset_index(drop=False)\
    .rename(columns={"index":"rn"}) # Create a row_num col based on the index of the data frame
    
    # Create a train_val_test field to denote which set each observation is a part of
    subset_df["train_val_test"] = np.where(subset_df["rn"]<(96*7*8),
                                           "train",
                                           np.where(subset_df["rn"]<(96*7*10),
                                                    "val",
                                                    "test"
                                                   )
                                       )

    # Append the subset df to the list
    subset_df_list.append(subset_df)

In [398]:
# Create two lists, one for the train_val data and one for the test data
train_df_list = list()
test_df_list = list()

# Loop through the list of subset dfs
for df in subset_df_list:
    
    # Filter to train and val only for the training data
    train_df = df.query("train_val_test != 'test'").copy()
    train_df = train_df[['timestamp', 'total_volume']]\
    .rename(columns={'timestamp':'ds', 'total_volume':'y'}) # For prophet, we must rename our timestamp column to ds and out target to y
    
    # Append to list
    train_df_list.append(train_df)
    
    # Same steps for test df list
    test_df = df.query("train_val_test == 'test'").copy()
    test_df = test_df[['timestamp', 'total_volume']]\
    .rename(columns={'timestamp':'ds', 'total_volume':'y'})
    test_df_list.append(test_df)

In [399]:
prediction_length = 1
context_length = 672

In [400]:
clust_assign = pd.read_csv("Results/Unseen Sensor/clust_assign.csv")

# Define Functions for Making Forecasts

In [402]:
# Code for progress bar:
# https://stackoverflow.com/questions/24983493/tracking-progress-of-joblib-parallel-execution
# This allows us to print a progress bar while running parallel loops using joblib 

@contextlib.contextmanager
def tqdm_joblib(tqdm_object):
    """Context manager to patch joblib to report into tqdm progress bar given as argument"""
    class TqdmBatchCompletionCallback(joblib.parallel.BatchCompletionCallBack):
        def __call__(self, *args, **kwargs):
            tqdm_object.update(n=self.batch_size)
            return super().__call__(*args, **kwargs)

    old_batch_callback = joblib.parallel.BatchCompletionCallBack
    joblib.parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback
    try:
        yield tqdm_object
    finally:
        joblib.parallel.BatchCompletionCallBack = old_batch_callback
        tqdm_object.close()

In [403]:
# Function to compute clustered test preds
def compute_deepar_test_preds_clust(model, test_df_ls, ts_idx_ls):
    """Function which takes inputs: a trained model, and a list of data frames which contain train+test data, 
    and which returns a df of model predictions on the test data"""
        
    forecast_means = dict()
    forecast_medians = dict()
    forecast_q10s = dict()
    forecast_q90s = dict()
    forecast_q025s = dict()
    forecast_q975s = dict()

    mod = model
    
    for idx in ts_idx_ls:
        forecast_means[idx] = list()
        forecast_medians[idx] = list()
        forecast_q10s[idx] = list()
        forecast_q90s[idx] = list()
        forecast_q025s[idx] = list()
        forecast_q975s[idx] = list()

    for n in range(1344):
        # print(n)
        eval_dict_ls = [{"start": df.iloc[0,5],
                         "target": df.iloc[:,[5,8]].iloc[0:(96*10*7+n),:].total_volume.values
                        } for df in test_df_ls]
        eval_data = ListDataset(eval_dict_ls, freq="15min")

        forecast_it, ts_it = make_evaluation_predictions(dataset=eval_data,  # test dataset
                                                         predictor=mod,  # predictor
                                                         num_samples=100,  # number of sample paths we want for evaluation
                                                        )

        forecasts = list(forecast_it)

        for i in range(len(forecasts)):
            forecast_mean = forecasts[i].samples.mean()
            forecast_means[ts_idx_ls[i]].append(forecast_mean)

            forecast_percentiles = np.percentile(forecasts[i].samples, [2.5, 10, 50, 90, 97.5])
            forecast_medians[ts_idx_ls[i]].append(forecast_percentiles[2])

            forecast_q10s[ts_idx_ls[i]].append(forecast_percentiles[1])
            forecast_q90s[ts_idx_ls[i]].append(forecast_percentiles[3])
            forecast_q025s[ts_idx_ls[i]].append(forecast_percentiles[0])
            forecast_q975s[ts_idx_ls[i]].append(forecast_percentiles[4])       
           
    return (forecast_means, forecast_medians, forecast_q10s, forecast_q90s, forecast_q025s, forecast_q975s)

In [405]:
# Create a function to compute the interval score
def interval_score(true_values, lower, upper, interval_range):
    """ Function which takes in the true values, the upper and lower bounds of PIs, and the PI level (e.g., 90%)
        and from these inputs, computes the interval score for each prediction
    """
    
    # Compute alpha from the interval range
    alpha = 100*(1-interval_range)
    
    # Save the upper, lower, and true_values as numpy arrays for computation purposes
    upper = np.array(upper)
    lower = np.array(lower)
    true_values = np.array(true_values)
    
    # Compute the lower component of the interval score - just a boolean for true below interval
    def lower_ind(true,low):
        if true<low:
            return 1
        else:
            return 0
        
    # Computer the upper component of the interval score - similar boolean for true above interval
    def upper_ind(true,up):
        if true>up:
            return 1
        else:
            return 0
        
    # Computer the actual score for each obsveration - formula here: https://epiforecasts.io/scoringutils/reference/interval_score.html
    scores = (upper-lower) + (2/alpha)*(lower-true_values)*(lower > true_values) + (2/alpha)*(true_values-upper)*(true_values > upper)
    
    # Return the scores array
    return scores

# Forecast with Global Model for All Data

In [401]:
with open('Results/Global/DeepAR/Full/mod', 'rb') as mod_file:
    mod = pickle.load(mod_file)

In [404]:
with tqdm_joblib(tqdm(desc="Full Model DeepAR Unseen Preds", 
                      total=(len(subset_df_list)))) as progress_bar:
    full_mod_preds = Parallel(n_jobs=4)(delayed(compute_deepar_test_preds_clust)(mod,
                                                                                 [subset_df_list[i]],
                                                                                 [i+1]
                                                                                ) for i in range(len(subset_df_list)))

Full Model DeepAR Unseen Preds: 100%|██████████| 4/4 [38:45<00:00, 581.27s/it]   


In [406]:
forec_means = {**full_mod_preds[0][0], 
               **full_mod_preds[1][0],
               **full_mod_preds[2][0], 
               **full_mod_preds[3][0]}

forec_lo_80 = {**full_mod_preds[0][2], 
               **full_mod_preds[1][2],
               **full_mod_preds[2][2], 
               **full_mod_preds[3][2]}

forec_hi_80 = {**full_mod_preds[0][3], 
               **full_mod_preds[1][3],
               **full_mod_preds[2][3], 
               **full_mod_preds[3][3]}

forec_lo_95 = {**full_mod_preds[0][4], 
               **full_mod_preds[1][4],
               **full_mod_preds[2][4], 
               **full_mod_preds[3][4]}

forec_hi_95 = {**full_mod_preds[0][5], 
               **full_mod_preds[1][5],
               **full_mod_preds[2][5], 
               **full_mod_preds[3][5]}


In [407]:
forec_actuals = {k: test_df_list[k-1].y.to_list() for k in range(1, (len(test_df_list)+1))}

forec_actual_means = {k: np.mean(test_df_list[k-1].y.to_list()) for k in range(1, (len(test_df_list)+1))}

In [408]:
full_int_score_80 = {k: interval_score(forec_actuals[k], 
                                       forec_lo_80[k],
                                       forec_hi_80[k],
                                       0.8) for k in forec_actuals.keys()
                    }

full_int_score_95 = {k: interval_score(forec_actuals[k], 
                                       forec_lo_95[k],
                                       forec_hi_95[k],
                                       0.95) for k in forec_actuals.keys()
                    }

full_rmse = {k: mean_squared_error(forec_actuals[k], 
                                   forec_means[k], 
                                   squared=False) for k in forec_actuals.keys()}

full_mae = {k: mean_absolute_error(forec_actuals[k], 
                                   forec_means[k]) for k in forec_actuals.keys()}

In [409]:
np.mean(list(full_rmse.values()))

57.98740754059675

In [410]:
np.mean(list(full_mae.values()))

38.8193564518754

In [411]:
np.mean([full_rmse[k]/forec_actual_means[k] for k in full_rmse.keys()])

0.15678632343074098

In [412]:
np.mean([full_mae[k]/forec_actual_means[k] for k in full_mae.keys()])

0.10843076351965276

In [413]:
np.mean(list(full_int_score_80.values()))

73.160084963087

In [414]:
np.mean(list(full_int_score_95.values()))

122.19913741105674

In [415]:
np.mean([full_int_score_80[k]/forec_actual_means[k] for k in full_int_score_80.keys()])

0.2394914899711956

In [416]:
np.mean([full_int_score_95[k]/forec_actual_means[k] for k in full_int_score_95.keys()])

0.39925932963070404

/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/gluonts/dataset/common.py:324: FutureWarning: The 'freq' argument in Timestamp is deprecated and will be removed in a future version.
  timestamp = pd.Timestamp(timestamp_input, freq=freq)
/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/gluonts/dataset/common.py:327: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version
  if isinstance(timestamp.freq, Tick):
/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/gluonts/dataset/common.py:329: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version
  timestamp.floor(timestamp.freq), timestamp.freq
/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/gluonts/dataset/common.py:328: FutureWarning: The 'freq' argument in Timestamp is deprecated and will be removed in a future version.
  return pd.Timestamp(
/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/gluon

# Forecast with Models for Random Clusters

In [34]:
clust_assign.rand

0    3
1    2
2    1
3    4
Name: rand, dtype: int64

In [418]:
with open(f'Results/Global/DeepAR/Random Cluster/mod_2', 'rb') as f:
    rand_clust_deep_ar_mod3 = pickle.load(f)
    
with open(f'Results/Global/DeepAR/Random Cluster/mod_1', 'rb') as f:
    rand_clust_deep_ar_mod2 = pickle.load(f)
    
with open(f'Results/Global/DeepAR/Random Cluster/mod_0', 'rb') as f:
    rand_clust_deep_ar_mod1 = pickle.load(f)
    
with open(f'Results/Global/DeepAR/Random Cluster/mod_3', 'rb') as f:
    rand_clust_deep_ar_mod4 = pickle.load(f)

In [419]:
rand_clust_deep_ar_mod_ls = [rand_clust_deep_ar_mod3, rand_clust_deep_ar_mod2, 
                             rand_clust_deep_ar_mod1, rand_clust_deep_ar_mod4]

In [420]:
with tqdm_joblib(tqdm(desc="Random Cluster DeepAR Unseen Preds", 
                      total=(len(subset_df_list)))) as progress_bar:
    rand_clust_test_preds = Parallel(n_jobs=4)(delayed(compute_deepar_test_preds_clust)(rand_clust_deep_ar_mod_ls[i],
                                                                                        [subset_df_list[i]],
                                                                                        [i+1]
                                                                                       ) for i in range(len(subset_df_list)))

Random Cluster DeepAR Unseen Preds: 100%|██████████| 4/4 [38:40<00:00, 580.17s/it]   


In [425]:
rand_clust_forec_means = {**rand_clust_test_preds[0][0], 
                          **rand_clust_test_preds[1][0],
                          **rand_clust_test_preds[2][0], 
                          **rand_clust_test_preds[3][0]}

rand_clust_forec_lo_80 = {**rand_clust_test_preds[0][2], 
                          **rand_clust_test_preds[1][2],
                          **rand_clust_test_preds[2][2], 
                          **rand_clust_test_preds[3][2]}

rand_clust_forec_hi_80 = {**rand_clust_test_preds[0][3], 
                          **rand_clust_test_preds[1][3],
                          **rand_clust_test_preds[2][3], 
                          **rand_clust_test_preds[3][3]}

rand_clust_forec_lo_95 = {**rand_clust_test_preds[0][4], 
                          **rand_clust_test_preds[1][4],
                          **rand_clust_test_preds[2][4], 
                          **rand_clust_test_preds[3][4]}

rand_clust_forec_hi_95 = {**rand_clust_test_preds[0][5], 
                          **rand_clust_test_preds[1][5],
                          **rand_clust_test_preds[2][5], 
                          **rand_clust_test_preds[3][5]}

rand_clust_forec_actuals = {k: test_df_list[k-1].y.to_list() for k in range(1, (len(test_df_list)+1))}

rand_clust_forec_actual_means = {k: np.mean(test_df_list[k-1].y.to_list()) for k in range(1, (len(test_df_list)+1))}

In [426]:
rand_clust_int_score_80 = {k: interval_score(rand_clust_forec_actuals[k], 
                                             rand_clust_forec_lo_80[k],
                                             rand_clust_forec_hi_80[k],
                                             0.8) for k in rand_clust_forec_actuals.keys()
                          }

rand_clust_int_score_95 = {k: interval_score(rand_clust_forec_actuals[k], 
                                             rand_clust_forec_lo_95[k],
                                             rand_clust_forec_hi_95[k],
                                             0.95) for k in rand_clust_forec_actuals.keys()
                          }

rand_clust_rmse = {k: mean_squared_error(rand_clust_forec_actuals[k], 
                                      rand_clust_forec_means[k], 
                                      squared=False) for k in rand_clust_forec_actuals.keys()}

rand_clust_mae = {k: mean_absolute_error(rand_clust_forec_actuals[k], 
                                      rand_clust_forec_means[k]) for k in rand_clust_forec_actuals.keys()}

In [427]:
np.mean(list(rand_clust_rmse.values()))

58.58108664894465

In [428]:
np.mean(list(rand_clust_mae.values()))

39.50346412669335

In [429]:
np.mean([rand_clust_rmse[k]/rand_clust_forec_actual_means[k] for k in rand_clust_rmse.keys()])

0.15873691747489932

In [430]:
np.mean([rand_clust_mae[k]/rand_clust_forec_actual_means[k] for k in rand_clust_mae.keys()])

0.10998238593431656

In [431]:
np.mean(list(rand_clust_int_score_80.values()))

75.27213428449862

In [432]:
np.mean(list(rand_clust_int_score_95.values()))

125.92690183867043

In [433]:
np.mean([rand_clust_int_score_80[k]/rand_clust_forec_actual_means[k] for k in rand_clust_int_score_80.keys()])

0.24896899777001802

In [434]:
np.mean([rand_clust_int_score_95[k]/rand_clust_forec_actual_means[k] for k in rand_clust_int_score_95.keys()])

0.4120750822706009

# Forecast with Highway System Model

In [281]:
with open(f'Results/Global/DeepAR/Highway System/mod_0', 'rb') as f:
    eng_mod = pickle.load(f)

In [ ]:
with tqdm_joblib(tqdm(desc="England Model DeepAR Unseen Preds", 
                      total=(len(subset_df_list)))) as progress_bar:
    eng_test_preds_total_ls = Parallel(n_jobs=4)(delayed(compute_deepar_test_preds_clust)(eng_mod,
                                                                                    [subset_df_list[i]],
                                                                                    [i+1]
                                                                                   ) for i in range(len(subset_df_list))) 


England Model DeepAR Unseen Preds:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
eng_forec_means = {**eng_test_preds_total_ls[0][0], 
                   **eng_test_preds_total_ls[1][0],
                   **eng_test_preds_total_ls[2][0], 
                   **eng_test_preds_total_ls[3][0]}

eng_forec_lo_80 = {**eng_test_preds_total_ls[0][2], 
                   **eng_test_preds_total_ls[1][2],
                   **eng_test_preds_total_ls[2][2], 
                   **eng_test_preds_total_ls[3][2]}

eng_forec_hi_80 = {**eng_test_preds_total_ls[0][3], 
                   **eng_test_preds_total_ls[1][3],
                   **eng_test_preds_total_ls[2][3], 
                   **eng_test_preds_total_ls[3][3]}

eng_forec_lo_95 = {**eng_test_preds_total_ls[0][4], 
                   **eng_test_preds_total_ls[1][4],
                   **eng_test_preds_total_ls[2][4], 
                   **eng_test_preds_total_ls[3][4]}

eng_forec_hi_95 = {**eng_test_preds_total_ls[0][5], 
                   **eng_test_preds_total_ls[1][5],
                   **eng_test_preds_total_ls[2][5], 
                   **eng_test_preds_total_ls[3][5]}

eng_forec_actuals = {k: test_df_list[k-1].y.to_list() for k in range(1, (len(test_df_list)+1))}

eng_forec_actual_means = {k: np.mean(test_df_list[k-1].y.to_list()) for k in range(1, (len(test_df_list)+1))}

In [ ]:
eng_int_score_80 = {k: interval_score(eng_forec_actuals[k], 
                                             eng_forec_lo_80[k],
                                             eng_forec_hi_80[k],
                                             0.8) for k in eng_forec_actuals.keys()
                          }

eng_int_score_95 = {k: interval_score(eng_forec_actuals[k], 
                                             eng_forec_lo_95[k],
                                             eng_forec_hi_95[k],
                                             0.95) for k in eng_forec_actuals.keys()
                          }

eng_rmse = {k: mean_squared_error(eng_forec_actuals[k], 
                                      eng_forec_means[k], 
                                      squared=False) for k in eng_forec_actuals.keys()}

eng_mae = {k: mean_absolute_error(eng_forec_actuals[k], 
                                      eng_forec_means[k]) for k in eng_forec_actuals.keys()}

In [ ]:
np.mean(list(eng_rmse.values()))

In [ ]:
np.mean(list(eng_mae.values()))

In [ ]:
np.mean([eng_rmse[k]/eng_forec_actual_means[k] for k in eng_rmse.keys()])

In [ ]:
np.mean([eng_mae[k]/eng_forec_actual_means[k] for k in eng_mae.keys()])

In [ ]:
np.mean(list(eng_int_score_80.values()))

In [ ]:
np.mean(list(eng_int_score_95.values()))

In [ ]:
np.mean([eng_int_score_80[k]/eng_forec_actual_means[k] for k in eng_int_score_80.keys()])

In [ ]:
np.mean([eng_int_score_95[k]/eng_forec_actual_means[k] for k in eng_int_score_95.keys()])

# Forecast with Models for Catch22-Based KMeans Clusters

In [341]:
clust_assign.catch22

0    1
1    1
2    1
3    1
Name: catch22, dtype: int64

In [343]:
with open(f'Results/Global/DeepAR/Catch22 KMeans/mod_0', 'rb') as f:
    catch22_mod1 = pickle.load(f)

In [350]:
with tqdm_joblib(tqdm(desc="Catch22 DeepAR Unseen Preds", 
                      total=(len(subset_df_list)))) as progress_bar:
    catch22_test_preds = Parallel(n_jobs=4)(delayed(compute_deepar_test_preds_clust)(catch22_mod1,
                                                                                    [subset_df_list[i]],
                                                                                    [i+1]
                                                                                   ) for i in range(len(subset_df_list))) 


Catch22 DeepAR Unseen Preds: 100%|██████████| 4/4 [38:29<00:00, 577.37s/it]   


In [351]:
catch22_forec_means = {**catch22_test_preds[0][0], 
                       **catch22_test_preds[1][0],
                       **catch22_test_preds[2][0], 
                       **catch22_test_preds[3][0]}

catch22_forec_lo_80 = {**catch22_test_preds[0][2], 
                       **catch22_test_preds[1][2],
                       **catch22_test_preds[2][2], 
                       **catch22_test_preds[3][2]}

catch22_forec_hi_80 = {**catch22_test_preds[0][3], 
                       **catch22_test_preds[1][3],
                       **catch22_test_preds[2][3], 
                       **catch22_test_preds[3][3]}

catch22_forec_lo_95 = {**catch22_test_preds[0][4], 
                       **catch22_test_preds[1][4],
                       **catch22_test_preds[2][4], 
                       **catch22_test_preds[3][4]}

catch22_forec_hi_95 = {**catch22_test_preds[0][5], 
                       **catch22_test_preds[1][5],
                       **catch22_test_preds[2][5], 
                       **catch22_test_preds[3][5]}

catch22_forec_actuals = {k: test_df_list[k-1].y.to_list() for k in range(1, (len(test_df_list)+1))}

catch22forec_actual_means = {k: np.mean(test_df_list[k-1].y.to_list()) for k in range(1, (len(test_df_list)+1))}

In [352]:
catch22_int_score_80 = {k: interval_score(catch22_forec_actuals[k], 
                                          catch22_forec_lo_80[k],
                                          catch22_forec_hi_80[k],
                                          0.8) for k in catch22_forec_actuals.keys()
                          }

catch22_int_score_95 = {k: interval_score(catch22_forec_actuals[k], 
                                          catch22_forec_lo_95[k],
                                          catch22_forec_hi_95[k],
                                          0.95) for k in catch22_forec_actuals.keys()
                          }

catch22_rmse = {k: mean_squared_error(catch22_forec_actuals[k], 
                                      catch22_forec_means[k], 
                                      squared=False) for k in catch22_forec_actuals.keys()}

catch22_mae = {k: mean_absolute_error(catch22_forec_actuals[k], 
                                      catch22_forec_means[k]) for k in catch22_forec_actuals.keys()}

In [353]:
np.mean(list(catch22_rmse.values()))

58.54068452681147

In [354]:
np.mean(list(catch22_mae.values()))

39.254392794289046

In [355]:
np.mean([catch22_rmse[k]/catch22forec_actual_means[k] for k in catch22_rmse.keys()])

0.15795296578300377

In [356]:
np.mean([catch22_mae[k]/catch22forec_actual_means[k] for k in catch22_mae.keys()])

0.10926380497163951

In [357]:
np.mean(list(catch22_int_score_80.values()))

72.64839381115934

In [358]:
np.mean(list(catch22_int_score_95.values()))

120.90600858029954

In [359]:
np.mean([catch22_int_score_80[k]/catch22forec_actual_means[k] for k in catch22_int_score_80.keys()])

0.2398174114750097

In [360]:
np.mean([catch22_int_score_95[k]/catch22forec_actual_means[k] for k in catch22_int_score_95.keys()])

0.39932288581426256

/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/gluonts/dataset/common.py:324: FutureWarning: The 'freq' argument in Timestamp is deprecated and will be removed in a future version.
  timestamp = pd.Timestamp(timestamp_input, freq=freq)
/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/gluonts/dataset/common.py:327: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version
  if isinstance(timestamp.freq, Tick):
/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/gluonts/dataset/common.py:329: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version
  timestamp.floor(timestamp.freq), timestamp.freq
/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/gluonts/dataset/common.py:328: FutureWarning: The 'freq' argument in Timestamp is deprecated and will be removed in a future version.
  return pd.Timestamp(
/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/gluon

# Forecast for Models with TSFeat-Based KMeans Clusters

In [361]:
clust_assign.tsfeat

0    2
1    2
2    2
3    1
Name: tsfeat, dtype: int64

In [362]:
with open(f'Results/Global/DeepAR/TSFeat KMeans/mod_1', 'rb') as f:
    tsfeat_mod2 = pickle.load(f)
    
with open(f'Results/Global/DeepAR/TSFeat KMeans/mod_0', 'rb') as f:
    tsfeat_mod1 = pickle.load(f)

In [363]:
ts_feat_mod_ls = [tsfeat_mod2, tsfeat_mod2, tsfeat_mod2, tsfeat_mod1]

In [365]:
with tqdm_joblib(tqdm(desc="TSFEat DeepAR Unseen Preds", 
                      total=(len(subset_df_list)))) as progress_bar:
    tsfeat_test_preds = Parallel(n_jobs=4)(delayed(compute_deepar_test_preds_clust)(ts_feat_mod_ls[i],
                                                                                    [subset_df_list[i]],
                                                                                    [i+1]
                                                                                   ) for i in range(len(subset_df_list))) 


TSFEat DeepAR Unseen Preds: 100%|██████████| 4/4 [38:27<00:00, 576.84s/it]   


In [366]:
tsfeat_forec_means = {**tsfeat_test_preds[0][0], 
                       **tsfeat_test_preds[1][0],
                       **tsfeat_test_preds[2][0], 
                       **tsfeat_test_preds[3][0]}

tsfeat_forec_lo_80 = {**tsfeat_test_preds[0][2], 
                       **tsfeat_test_preds[1][2],
                       **tsfeat_test_preds[2][2], 
                       **tsfeat_test_preds[3][2]}

tsfeat_forec_hi_80 = {**tsfeat_test_preds[0][3], 
                       **tsfeat_test_preds[1][3],
                       **tsfeat_test_preds[2][3], 
                       **tsfeat_test_preds[3][3]}

tsfeat_forec_lo_95 = {**tsfeat_test_preds[0][4], 
                       **tsfeat_test_preds[1][4],
                       **tsfeat_test_preds[2][4], 
                       **tsfeat_test_preds[3][4]}

tsfeat_forec_hi_95 = {**tsfeat_test_preds[0][5], 
                       **tsfeat_test_preds[1][5],
                       **tsfeat_test_preds[2][5], 
                       **tsfeat_test_preds[3][5]}

tsfeat_forec_actuals = {k: test_df_list[k-1].y.to_list() for k in range(1, (len(test_df_list)+1))}

tsfeatforec_actual_means = {k: np.mean(test_df_list[k-1].y.to_list()) for k in range(1, (len(test_df_list)+1))}

In [367]:
tsfeat_int_score_80 = {k: interval_score(tsfeat_forec_actuals[k], 
                                          tsfeat_forec_lo_80[k],
                                          tsfeat_forec_hi_80[k],
                                          0.8) for k in tsfeat_forec_actuals.keys()
                          }

tsfeat_int_score_95 = {k: interval_score(tsfeat_forec_actuals[k], 
                                          tsfeat_forec_lo_95[k],
                                          tsfeat_forec_hi_95[k],
                                          0.95) for k in tsfeat_forec_actuals.keys()
                          }

tsfeat_rmse = {k: mean_squared_error(tsfeat_forec_actuals[k], 
                                      tsfeat_forec_means[k], 
                                      squared=False) for k in tsfeat_forec_actuals.keys()}

tsfeat_mae = {k: mean_absolute_error(tsfeat_forec_actuals[k], 
                                      tsfeat_forec_means[k]) for k in tsfeat_forec_actuals.keys()}

In [368]:
np.mean(list(tsfeat_rmse.values()))

58.421084501519516

In [369]:
np.mean(list(tsfeat_mae.values()))

39.15330388893683

In [370]:
np.mean([tsfeat_rmse[k]/tsfeatforec_actual_means[k] for k in tsfeat_rmse.keys()])

0.1581634011767009

In [371]:
np.mean([tsfeat_mae[k]/tsfeatforec_actual_means[k] for k in tsfeat_mae.keys()])

0.10943409379408638

In [372]:
np.mean(list(tsfeat_int_score_80.values()))

76.16986844125293

In [373]:
np.mean(list(tsfeat_int_score_95.values()))

127.28346461222708

In [374]:
np.mean([tsfeat_int_score_80[k]/tsfeatforec_actual_means[k] for k in tsfeat_int_score_80.keys()])

0.2510172030112627

In [375]:
np.mean([tsfeat_int_score_95[k]/tsfeatforec_actual_means[k] for k in tsfeat_int_score_95.keys()])

0.41849294734980785

# Forecast with Models for DTW-Based K-Medoids Clusters

In [376]:
clust_assign.dtw

0    2
1    1
2    1
3    1
Name: dtw, dtype: int64

In [377]:
with open(f'Results/Global/DeepAR/DTW/mod_1', 'rb') as f:
    dtw_mod1 = pickle.load(f)
    
with open(f'Results/Global/DeepAR/DTW/mod_0', 'rb') as f:
    dtw_mod1 = pickle.load(f)

In [378]:
dtw_mod_list = [dtw_mod2, dtw_mod1, dtw_mod1, dtw_mod1]

In [379]:
with tqdm_joblib(tqdm(desc="DTW DeepAR Unseen Preds", 
                      total=(len(subset_df_list)))) as progress_bar:
    dtw_test_preds = Parallel(n_jobs=4)(delayed(compute_deepar_test_preds_clust)(dtw_mod_list[i],
                                                                                 [subset_df_list[i]],
                                                                                 [i+1]
                                                                                ) for i in range(len(subset_df_list))) 


DTW DeepAR Unseen Preds: 100%|██████████| 4/4 [39:01<00:00, 585.31s/it]   


In [380]:
dtw_forec_means = {**dtw_test_preds[0][0], 
                       **dtw_test_preds[1][0],
                       **dtw_test_preds[2][0], 
                       **dtw_test_preds[3][0]}

dtw_forec_lo_80 = {**dtw_test_preds[0][2], 
                       **dtw_test_preds[1][2],
                       **dtw_test_preds[2][2], 
                       **dtw_test_preds[3][2]}

dtw_forec_hi_80 = {**dtw_test_preds[0][3], 
                       **dtw_test_preds[1][3],
                       **dtw_test_preds[2][3], 
                       **dtw_test_preds[3][3]}

dtw_forec_lo_95 = {**dtw_test_preds[0][4], 
                       **dtw_test_preds[1][4],
                       **dtw_test_preds[2][4], 
                       **dtw_test_preds[3][4]}

dtw_forec_hi_95 = {**dtw_test_preds[0][5], 
                       **dtw_test_preds[1][5],
                       **dtw_test_preds[2][5], 
                       **dtw_test_preds[3][5]}

dtw_forec_actuals = {k: test_df_list[k-1].y.to_list() for k in range(1, (len(test_df_list)+1))}

dtwforec_actual_means = {k: np.mean(test_df_list[k-1].y.to_list()) for k in range(1, (len(test_df_list)+1))}

In [381]:
dtw_int_score_80 = {k: interval_score(dtw_forec_actuals[k], 
                                          dtw_forec_lo_80[k],
                                          dtw_forec_hi_80[k],
                                          0.8) for k in dtw_forec_actuals.keys()
                          }

dtw_int_score_95 = {k: interval_score(dtw_forec_actuals[k], 
                                          dtw_forec_lo_95[k],
                                          dtw_forec_hi_95[k],
                                          0.95) for k in dtw_forec_actuals.keys()
                          }

dtw_rmse = {k: mean_squared_error(dtw_forec_actuals[k], 
                                      dtw_forec_means[k], 
                                      squared=False) for k in dtw_forec_actuals.keys()}

dtw_mae = {k: mean_absolute_error(dtw_forec_actuals[k], 
                                      dtw_forec_means[k]) for k in dtw_forec_actuals.keys()}

In [382]:
np.mean(list(dtw_rmse.values()))

57.679165966430624

In [383]:
np.mean(list(dtw_mae.values()))

38.8368039149791

In [384]:
np.mean([dtw_rmse[k]/dtwforec_actual_means[k] for k in dtw_rmse.keys()])

0.15801515765796714

In [385]:
np.mean([dtw_mae[k]/dtwforec_actual_means[k] for k in dtw_mae.keys()])

0.1091288398614969

In [386]:
np.mean(list(dtw_int_score_80.values()))

73.41025860568756

In [387]:
np.mean(list(dtw_int_score_95.values()))

124.77837883467878

In [388]:
np.mean([dtw_int_score_80[k]/dtwforec_actual_means[k] for k in dtw_int_score_80.keys()])

0.2358479553162288

In [389]:
np.mean([dtw_int_score_95[k]/dtwforec_actual_means[k] for k in dtw_int_score_95.keys()])

0.3999440920276461

/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/gluonts/dataset/common.py:324: FutureWarning: The 'freq' argument in Timestamp is deprecated and will be removed in a future version.
  timestamp = pd.Timestamp(timestamp_input, freq=freq)
/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/gluonts/dataset/common.py:327: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version
  if isinstance(timestamp.freq, Tick):
/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/gluonts/dataset/common.py:329: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version
  timestamp.floor(timestamp.freq), timestamp.freq
/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/gluonts/dataset/common.py:328: FutureWarning: The 'freq' argument in Timestamp is deprecated and will be removed in a future version.
  return pd.Timestamp(
/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/gluon